In [ ]:
#Use this code whenever we've updated the environment.yml document
#%conda env update --file '../environment.yml'  --prune

In [1]:
import scipy as sp
import numpy as np
import os
from tqdm.auto import tqdm
from hurry.filesize import size

from skimage.segmentation import expand_labels

from dask_image import ndmeasure
from dask import array as da
from dask.distributed import Client, progress

import napari

# Expand Finger Labels and Save Individual Fingers

This notebook takes a dask array of labeled bones in the hand for a relatively high-threshold so that metacarpals and proximal phalanges are distinct.  It then expands those labels so that they do not overlab until they inlude all of the interior of each finger.  It then uses those expanded labels as masks to save the individual fingers to their own files.  The fingers must be labeled such that index = 1, middle = 2, ring = 3, and pinky = 4.

In [2]:
client = Client(processes=False, threads_per_worker=6,
                n_workers=1, memory_limit='12GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.19.111.216:8787/status,
Dashboard: http://172.19.111.216:8787/status,Workers: 1
Total threads: 6,Total memory: 11.18 GiB
Status: running,Using processes: False
Comm: inproc://172.19.111.216/2384/1,Workers: 0
Dashboard: http://172.19.111.216:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: inproc://172.19.111.216/2384/4,Total threads: 6
Dashboard: http://172.19.111.216:57379/status,Memory: 11.18 GiB
Nanny: None,


### User Input

These are the user inputs needed for this algorithm to work properly.  Specifically:
 - `specific` is the name of the folder where the input data is located.
 - `d` will be the depth of the label blocks to use when we expand the labels.
 - `num_rounds_expand_labels` will be the number of iterations that we expand the labels for a distance of $\dfrac{d}{2}$.

In [3]:
specific = 'F231006L'
d = 20
num_rounds_expand_labels = 4

Next, we define the various directories we will need, and we will create them if they do not yet already exist.

In [4]:
hand_dir = '../hands/' + specific + '/'
temp_finger_labels = '../hands/' + specific + '_temp_finger_labels/'
temp_block_labels = '../hands/' + specific + '_temp_finger_blocks/'
temp_dir = 'temp/'
finger_dir = {1: '../fingers/index/full/' + specific + '/',
              2: '../fingers/middle/full/' + specific + '/',
              3: '../fingers/ring/full/' + specific + '/',
              4: '../fingers/pinky/full/' + specific + '/',
             }
for d in [temp_dir, temp_finger_labels, temp_block_labels] + list(finger_dir.values()):
    os.makedirs(d, exist_ok=True)

In [5]:
hand = da.from_npy_stack(hand_dir)
labels = da.from_npy_stack(temp_finger_labels)
spacing = np.load('../spacing/' + specific + '.npy')

In [6]:
labels = da.from_npy_stack(temp_finger_labels)

In [7]:
labels

dask.array<from-npy-stack, shape=(1278, 1335, 1757), dtype=int8, chunksize=(57, 1335, 1757), chunktype=numpy.ndarray>

In [9]:
sheet_size = labels.chunksize
block_size = (d,) + labels.chunksize[1:]

In [10]:
labels = labels.rechunk(block_size)

In [11]:
labels

dask.array<rechunk-merge, shape=(1278, 1335, 1757), dtype=int8, chunksize=(20, 1335, 1757), chunktype=numpy.ndarray>

In [12]:
da.to_npy_stack(temp_block_labels, labels)

In [13]:
labels = da.from_npy_stack(temp_block_labels)

In [14]:
def empty_the_folder(directory):
    #This method deletes all files from a temporary folder.
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)
    print(f'removed files from {directory}')

In [15]:
def fill_the_folder(directory):
    #This methods fills a temporary folder with a copy of labels and returns the number of files
    temp = labels.copy()
    da.to_npy_stack(directory, temp)
    print(f'filled {directory} with copy of labels')
    return temp.blocks.size

In [16]:
def before_current_after(i, n, old_current, old_after, directory):
    #This method is for iterating through a stack of numpy files.
    #It returns the current numpy file and it's two neighbors.
    if i == 0:
        current = np.load(directory + str(i) + '.npy')
        after = np.load(directory + str(i+1) + '.npy')
        before = np.array([], dtype=current.dtype).reshape((0,) + current.shape[1:])
    elif i < n - 1:
        before = old_current
        current = old_after
        after = np.load(directory + str(i+1) + '.npy')
    else:
        before = old_current
        current = old_after
        after = np.array([], dtype=current.dtype).reshape((0,) + current.shape[1:])
    return before, current, after

In [17]:
def expand_labels_blocks(before, current, after):
    #This function iterates through an .npy stack of labels and expands the labels by d//2 voxels
    #where d is the depth of the numpy arrays.  It only saves the current block, not it's neighbors.
    big = np.concatenate((before, current, after))
    keep = np.arange(before.shape[0], before.shape[0]+current.shape[0])
    return expand_labels(big, distance=d//2)[keep]

In [18]:
#First, make sure the temporary folder is empty
empty_the_folder(temp_dir)
for j in range(num_rounds_expand_labels):
    print(f'round {j+1} of {num_rounds_expand_labels} at distance {d//2}')
    #Populate the temp folder with a copy of labels, keeping track of the number of files: n.
    n = fill_the_folder(temp_dir)
    #For each block we are going to apply the numpy function expand_labels to it and its two neighbors
    current, after = [None, None] #Not needed for the first iteration
    for i in tqdm(range(n)):
        #Load the block and it's two neighbors
        before, current, after = before_current_after(i, n, current, after, temp_dir)
        #Expand the labels d//2 where d is the depth of the blocks
        current_expanded = expand_labels_blocks(before, current, after)
        #Save the newly expanded labels block in the folder where labels came from
        np.save(f'{temp_block_labels}{i}.npy', current_expanded)
    #Remove all of the temporary files that were copies of the old labels
    empty_the_folder(temp_dir)
    #Reload the labels from the newly changed files
    labels = da.from_npy_stack(temp_block_labels)
    print('Reloaded labels from the newly changed files.')

removed files from temp/
round 1 of 4 at distance 10
filled temp/ with copy of labels


  0%|          | 0/64 [00:00<?, ?it/s]

removed files from temp/
Reloaded labels from the newly changed files.
round 2 of 4 at distance 10
filled temp/ with copy of labels


  0%|          | 0/64 [00:00<?, ?it/s]

removed files from temp/
Reloaded labels from the newly changed files.
round 3 of 4 at distance 10
filled temp/ with copy of labels


  0%|          | 0/64 [00:00<?, ?it/s]

removed files from temp/
Reloaded labels from the newly changed files.
round 4 of 4 at distance 10
filled temp/ with copy of labels


  0%|          | 0/64 [00:00<?, ?it/s]

removed files from temp/
Reloaded labels from the newly changed files.


To double check that we have expanded our labels enough, we can use the histogram of label sizes.  There ought to be 2 large, and there could be a smattering of 1's.  Any other numbers might be a hole in one of the labels.

In [19]:
holes_labels, num_holes = ndmeasure.label(labels==0)

In [20]:
da.to_npy_stack(temp_dir, holes_labels)

In [21]:
holes_labels = da.from_npy_stack(temp_dir)

In [22]:
counts = da.bincount(holes_labels.ravel()).compute()
print(counts)

[ 196613116 2801053195          1          1          1          1
          1          1       3610          1         52          1
          2          2          1          2          1          1
          1          3          1          1          1          1
          2          1          1          1        287          1
          1          1          1          1          1          1
          1          2          2          1          1          1
          1          1          2          2          2          1
          1          1          1          1          1         13
          3          2          3          3          2          2
          3          2          2          2          1          2
          1          2          3          1          1          2
          4          2          4          1          1          2
          2          1          1          2          1          1
          1          1          1          1          1       

In [23]:
len(counts)

99

In [24]:
empty_the_folder(temp_dir)

removed files from temp/


Another way to double-check is to view the expanded labels with Napari:

In [25]:
viewer = napari.Viewer()
hand_view = viewer.add_image(
    hand,
    scale=spacing,
)
viewer.add_labels(
    labels,
    name='fingers_expanded',
    opacity=1.0,
    scale=spacing,
)

<Labels layer 'fingers_expanded' at 0x20c90551550>

In [26]:
for i in range(1,5):
    sub_x, sub_y, sub_z = da.where(labels == i)
    
    sub_x = sub_x.compute()
    x_min = sub_x.min()
    x_max = sub_x.max()
    del sub_x
    
    sub_y = sub_y.compute()
    y_min = sub_y.min()
    y_max = sub_y.max()
    del sub_y
    
    sub_z = sub_z.compute()
    z_min = sub_z.min()
    z_max = sub_z.max()
    del sub_z
    
    subset = da.where(labels == i, hand, -1000)[x_min:x_max, y_min:y_max, z_min:z_max]
    da.to_npy_stack(finger_dir[i], subset)
    
    print(f'x from {x_min} to {x_max}')
    print(f'y from {y_min} to {y_max}')
    print(f'z from {z_min} to {z_max}')
    print(f'Size of finger {i} array: {size(subset.nbytes)}')
    print('_'*20)

x from 185 to 1228
y from 403 to 803
z from 684 to 997
Size of finger 1 array: 996M
____________________
x from 192 to 1277
y from 402 to 802
z from 874 to 1233
Size of finger 2 array: 1G
____________________
x from 123 to 1226
y from 404 to 755
z from 1071 to 1436
Size of finger 3 array: 1G
____________________
x from 59 to 951
y from 408 to 699
z from 1295 to 1630
Size of finger 4 array: 663M
____________________


Below, you can use Napari to double check how each finger looks all by itself.

In [ ]:
i = 1
finger = da.from_npy_stack(finger_dir[i])
viewer = napari.Viewer()
viewer.add_image(finger)